# Localized Homicide Clearance Rates, 1995-present


In [2]:
import os
import sqlite3
import pandas as pd


## Data Import

In [3]:
def should_chdir():
    """checks whether the code is being run interactively or from command line"""
    # if both these directories exist, treat is as the correct directory
    dirs = ["input", "output"]
    if all(os.path.exists(d) for d in dirs):
        return False

    try:
        shell = get_ipython().__class__.__name__  # type: ignore
        if shell == "ZMQInteractiveShell":
            return True
        elif shell == "TerminalInteractiveShell":
            return False
        else:
            return False
    except NameError:
        return False


# step backwards if notebook is being run interactively
# or stay put if being run from command line
if should_chdir():
    os.chdir("..")


In [4]:
with sqlite3.connect("input/sqlite__temp.db") as con:
    agencies = pd.read_sql("SELECT * FROM data_agencies;", con)
    gb_cols = "ori_code, agency_name, card, year"
    reta = pd.read_sql(
        f"""
        SELECT {gb_cols}, SUM(value) as total
        FROM data_reta
        GROUP BY {gb_cols};
    """,
        con,
    )


### Merging geographies

In [5]:
df = reta.merge(
    agencies,
    how="left",
    left_on=["ori_code", "year"],
    right_on=["ori", "data_year"],
)


In [34]:
# FDLK24000 is for federal agencies, which all use the same ORI and won't be included here
df = df[df.ori_code != "FDLK24000"]

# TODO: delete this once you integrate it into the pipeline in transform task
# df[df.data_year == 2017].sort_values("total", ascending=False)
df = df[df.unique_id != "8735674e78d06fe3b1ea599b5d2ef22a018a6fb1"]

There are a number of records in which there are multiple agencies associated with the same ORI. This includes larger agencies like the Las Vegas Police Department, which I want to include in this analysis. 

In [35]:
def get_dupe_oris(d):
    """
    returns a dataframe with the ori codes and years for
    which there are multiple agency names with the same ORI code
    """
    return (
        d.groupby(["ori_code", "year"])
        .agency_name.nunique()
        .to_frame("n")
        .query("n > 1")
        .sort_values("n", ascending=False)
    )


dupe_oris = get_dupe_oris(df)
dupe_oris


n
ori_code  year   
CA0340200 2017  2
NJ0040700 2019  2
OK0160100 2017  2
NY0300100 2020  2
          2019  2
...            ..
KY0340100 2017  2
KS0890400 2020  2
          2019  2
          2018  2
VADI00100 2020  2

[85 rows x 1 columns]

## Analysis

In [37]:
def get_clearance_rate(crosstab_df):
    for year in df.year.unique().tolist():
        crosstab_df[(year, "clearance_rate")] = (
            crosstab_df[(year, "cleared_arrest")] / crosstab_df[(year, "actual")]
        )
    crosstab_df = crosstab_df.sort_index(axis=1, level="year", sort_remaining=False)
    return crosstab_df


def get5yravg_vs_2020(clearance_rate_df):
    cr_5yravg = clearance_rate_df[
        [(yr, "clearance_rate") for yr in range(2015, 2021)]
    ].droplevel(1, axis=1)
    cr_5yravg["5_year_avg"] = cr_5yravg.iloc[:, :-1].mean(axis=1)
    cr_5yravg = cr_5yravg[["5_year_avg", 2020]]
    cr_5yravg["change"] = (cr_5yravg[2020] - cr_5yravg["5_year_avg"]) / cr_5yravg[
        "5_year_avg"
    ]
    return cr_5yravg


### National

In [89]:
pd.crosstab(
    index=df.year,
    columns=df.card,
    values=df.total,
    aggfunc=sum,
).assign(clearance_rate=lambda df: df.cleared_arrest / df.actual)


card,actual,cleared_arrest,clearance_rate
year,,,
1995,20365,11766,0.577756
1996,17977,10531,0.585804
1997,16930,10286,0.607561
1998,15749,9866,0.626452
1999,14361,8991,0.626071
2000,14728,8598,0.583786
2001,15316,8746,0.571037
2002,15617,9099,0.582634
2003,15814,8849,0.559567


### By state

In [93]:
(
    pd.crosstab(
        index=[df.year, df.state_abbr],
        columns=df.card,
        values=df.total,
        aggfunc=sum,
    )
    .assign(clearance_rate=lambda df: df.cleared_arrest / df.actual)
    .stack()
    .to_frame("year")
    .unstack("state_abbr")
    .transpose()
    .droplevel(0)
)


year          1995                                  1996                 \
card        actual cleared_arrest clearance_rate  actual cleared_arrest   
state_abbr                                                                
AK            53.0           39.0       0.735849    43.0           33.0   
AL           466.0          127.0       0.272532   438.0          106.0   
AR           259.0          203.0       0.783784   218.0          181.0   
AS             NaN            NaN            NaN     NaN            NaN   
AZ           429.0          254.0       0.592075   368.0          217.0   
CA          3324.0         1819.0       0.547232  2851.0         1695.0   
CO           212.0          160.0       0.754717   176.0          121.0   
CT           150.0          121.0       0.806667   158.0          115.0   
CZ             0.0            0.0            NaN     0.0            0.0   
DC           361.0          137.0       0.379501   397.0           95.0   
DE             9.0            8.0       0.888889    29.0           19.0   
FL          1027.0          635.0       0.618306   581.0            0.0   
FS             0.0            0.0            NaN     0.0            0.0   
GA           657.0          524.0       0.797565   606.0          463.0   
GM             0.0            0.0            NaN     0.0            0.0   
HI            56.0           47.0       0.839286    40.0           35.0   
IA            54.0           21.0       0.388889    58.0           37.0   
ID            48.0           35.0       0.729167    43.0           34.0   
IL           880.0            0.0       0.000000   866.0            0.0   
IN           405.0          179.0       0.441975   369.0          183.0   
KS            50.0            0.0       0.000000    41.0            0.0   
KY           267.0          200.0       0.749064    97.0           63.0   
LA           728.0          453.0       0.622253   726.0          354.0   
MA           206.0          127.0       0.616505   157.0          100.0   
MD           596.0          329.0       0.552013   591.0          358.0   
ME            22.0           23.0       1.045455    25.0           20.0   
MI           785.0          177.0       0.225478   690.0          414.0   
MN           179.0           70.0       0.391061   171.0           65.0   
MO           436.0          370.0       0.848624   391.0          297.0   
MP             NaN            NaN            NaN     NaN            NaN   
MS           207.0          125.0       0.603865   199.0          134.0   
MT            10.0            7.0       0.700000     0.0            0.0   
NC           667.0          540.0       0.809595   612.0          481.0   
ND             6.0            3.0       0.500000    10.0            7.0   
NE            48.0           44.0       0.916667    47.0           41.0   
NH            18.0           13.0       0.722222    18.0           19.0   
NJ           409.0          332.0       0.811736   339.0          281.0   
NM           109.0           45.0       0.412844   139.0           57.0   
NV           162.0           98.0       0.604938   219.0           98.0   
NY          1535.0          588.0       0.383062  1330.0          862.0   
OH           543.0          299.0       0.550645   484.0          297.0   
OK           398.0          367.0       0.922111   223.0          189.0   
OR           125.0           76.0       0.608000   125.0           80.0   
OT             NaN            NaN            NaN     NaN            NaN   
PA           725.0          521.0       0.718621   678.0          504.0   
PR             0.0            0.0            NaN     0.0            0.0   
RI            33.0           10.0       0.303030    25.0           13.0   
SC           292.0          196.0       0.671233   335.0          270.0   
SD            12.0            9.0       0.750000     6.0            6.0   
TN           474.0          267.0       0.563291   408.0          288.0   
TX          1692.0       

### By MSA

In [91]:
msa = get_clearance_rate(
    pd.crosstab(
        index=[df.msa_name],
        columns=[df.year, df.card],
        values=df.total,
        aggfunc=sum,
    )
)

msa


year                                1995                                 1996  \
card                              actual cleared_arrest clearance_rate actual   
msa_name                                                                        
Abilene, TX                          5.0            2.0       0.400000    8.0   
Aguadilla-Isabela, PR                NaN            NaN            NaN    NaN   
Akron, OH                           22.0           18.0       0.818182   20.0   
Albany, GA                          16.0           15.0       0.937500    3.0   
Albany-Lebanon, OR                   NaN            NaN            NaN    NaN   
...                                  ...            ...            ...    ...   
Yakima, WA                          13.0            8.0       0.615385   16.0   
York-Hanover, PA                     9.0            8.0       0.888889    5.0   
Youngstown-Warren-Boardman, OH-PA   72.0            0.0       0.000000   62.0   
Yuba City, CA                        6.0            5.0       0.833333    7.0   
Yuma, AZ                             3.0            2.0       0.666667    4.0   

year                                                              1997  \
card                              cleared_arrest clearance_rate actual   
msa_name                                                                 
Abilene, TX                                  6.0       0.750000    2.0   
Aguadilla-Isabela, PR                        NaN            NaN    NaN   
Akron, OH                                    3.0       0.150000    4.0   
Albany, GA                                   3.0       1.000000   10.0   
Albany-Lebanon, OR                           NaN            NaN    NaN   
...                                          ...            ...    ...   
Yakima, WA                                  12.0       0.750000   15.0   
York-Hanover, PA                             4.0       0.800000    4.0   
Youngstown-Warren-Boardman, OH-PA           16.0       0.258065   46.0   
Yuba City, CA                                3.0       0.428571   10.0   
Yuma, AZ                                     1.0       0.250000    1.0   

year                                                              1998  ...  \
card                              cleared_arrest clearance_rate actual  ...   
msa_name                                                                ...   
Abilene, TX                                  2.0       1.000000    5.0  ...   
Aguadilla-Isabela, PR                        NaN            NaN    NaN  ...   
Akron, OH                                    3.0       0.750000    1.0  ...   
Albany, GA                                   8.0       0.800000   10.0  ...   
Albany-Lebanon, OR                           NaN            NaN    NaN  ...   
...                                          ...            ...    ...  ...   
Yakima, WA                                  10.0       0.666667   10.0  ...   
York-Hanover, PA                             3.0       0.750000   11.0  ...   
Youngstown-Warren-Boardman, OH-PA            0.0       0.000000   50.0  ...   
Yuba City, CA                                9.0       0.900000    7.0  ...   
Yuma, AZ                                     1.0       1.000000    0.0  ...   

year                                        2017   2018                 \
card                              clearance_rate actual cleared_arrest   
msa_name                                                                 
Abilene, TX                             0.857143    9.0            7.0   
Aguadilla-Isabela, PR                        NaN    0.0            0.0   
Akron, OH                               0.660377   44.0           28.0   
Albany, GA                              0.913043   17.0           14.0   
Albany-Lebanon, OR                      0.500000    1.0            2.0   
...                                          ...    ...            ...   
Yakima, WA                              0.380952   25.0

In [92]:
msa_5yravg = get5yravg_vs_2020(msa)
msa_5yravg


year,5_year_avg,2020,change
msa_name,,,
"Abilene, TX",0.911833,0.833333,-0.086090
"Aguadilla-Isabela, PR",NaN,NaN,NaN
"Akron, OH",0.616586,0.727273,0.179516
"Albany, GA",0.845799,0.565217,-0.331736
"Albany-Lebanon, OR",1.025000,0.400000,-0.609756
...,...,...,...
"Yakima, WA",0.549073,0.761905,0.387621
"York-Hanover, PA",0.509012,0.000000,-1.000000
"Youngstown-Warren-Boardman, OH-PA",0.232576,0.333333,0.433224


### By agency

In [40]:
agency = get_clearance_rate(
    pd.crosstab(
        index=[df.ori_code, df.agency_name],
        columns=[df.year, df.card],
        values=df.total,
        aggfunc=sum,
    )
)
agency


year                                 1995                                \
card                               actual cleared_arrest clearance_rate   
ori_code  agency_name                                                     
AK0010100 ANCHORAGE                  29.0           19.0       0.655172   
AK0010200 FAIRBANKS                   2.0            2.0       1.000000   
AK0010300 JUNEAU                      0.0            0.0            NaN   
AK0010400 KETCHIKAN                   0.0            0.0            NaN   
AK0010500 KODIAK                      0.0            0.0            NaN   
...                                   ...            ...            ...   
WYDI00600 BIGHORN CANYON NRA          NaN            NaN            NaN   
WYDI05000 WIND RIVER AGENCY           NaN            NaN            NaN   
          WIND RIVER TRIBAL           NaN            NaN            NaN   
WYWHP0000 WYOMING HIGHWAY PATROL      0.0            0.0            NaN   
WYWHP0100 HIGHWAY PATROL, CHEYENNE    NaN            NaN            NaN   

year                                 1996                                \
card                               actual cleared_arrest clearance_rate   
ori_code  agency_name                                                     
AK0010100 ANCHORAGE                  25.0           16.0           0.64   
AK0010200 FAIRBANKS                   3.0            3.0           1.00   
AK0010300 JUNEAU                      0.0            0.0            NaN   
AK0010400 KETCHIKAN                   0.0            0.0            NaN   
AK0010500 KODIAK                      0.0            0.0            NaN   
...                                   ...            ...            ...   
WYDI00600 BIGHORN CANYON NRA          NaN            NaN            NaN   
WYDI05000 WIND RIVER AGENCY           NaN            NaN            NaN   
          WIND RIVER TRIBAL           NaN            NaN            NaN   
WYWHP0000 WYOMING HIGHWAY PATROL      0.0            0.0            NaN   
WYWHP0100 HIGHWAY PATROL, CHEYENNE    NaN            NaN            NaN   

year                                 1997                                \
card                               actual cleared_arrest clearance_rate   
ori_code  agency_name                                                     
AK0010100 ANCHORAGE                  23.0           21.0       0.913043   
AK0010200 FAIRBANKS                   6.0            5.0       0.833333   
AK0010300 JUNEAU                      0.0            0.0            NaN   
AK0010400 KETCHIKAN                   0.0            0.0            NaN   
AK0010500 KODIAK                      0.0            0.0            NaN   
...                                   ...            ...            ...   
WYDI00600 BIGHORN CANYON NRA          NaN            NaN            NaN   
WYDI05000 WIND RIVER AGENCY           NaN            NaN            NaN   
          WIND RIVER TRIBAL           NaN            NaN            NaN   
WYWHP0000 WYOMING HIGHWAY PATROL      0.0            0.0            NaN   
WYWHP0100 HIGHWAY PATROL, CHEYENNE    NaN            NaN            NaN   

year                                 1998  ...           2017   2018  \
card                               actual  ... clearance_rate actual   
ori_code  agency_name                      ...                         
AK0010100 ANCHORAGE                  19.0  ...       1.037037   26.0   
AK0010200 FAIRBANKS                   4.0  ...       0.600000    4.0   
AK0010300 JUNEAU                      0.0  ...       3.000000    1.0   
AK0010400 KETCHIKAN                   0.0  ...       1.000000    1.0   
AK0010500 KODIAK                      0.0  ...            NaN    0.0   
...                                   ...  ...            ...    ...   
WYDI00600 BIGHORN CANYON NRA          NaN  ...            NaN    0.0   
WYDI05000 WIND RIVER AGENCY           NaN  ...       1.000000    1.0   
          WIND RIVER TRIBAL           NaN  ...         

In [41]:
agency_5yravg = get5yravg_vs_2020(agency)
agency_5yravg


year                                5_year_avg      2020    change
ori_code  agency_name                                             
AK0010100 ANCHORAGE                   0.750745  0.833333  0.110008
AK0010200 FAIRBANKS                   0.910476  1.000000  0.098326
AK0010300 JUNEAU                           inf  2.000000       NaN
AK0010400 KETCHIKAN                   1.000000  1.000000  0.000000
AK0010500 KODIAK                      1.000000       NaN       NaN
...                                        ...       ...       ...
WYDI00600 BIGHORN CANYON NRA               NaN       NaN       NaN
WYDI05000 WIND RIVER AGENCY           0.750000       NaN       NaN
          WIND RIVER TRIBAL                NaN       NaN       NaN
WYWHP0000 WYOMING HIGHWAY PATROL           NaN       NaN       NaN
WYWHP0100 HIGHWAY PATROL, CHEYENNE         NaN       NaN       NaN

[29735 rows x 3 columns]

### Selecting only "core" agencies

In [42]:
AGENCIES = [
    "CA0194200",
    "CA0380100",
    "CA0340400",
    "TXDPD0000",
    "TX2201200",
    "TXHPD0000",
    "FL0130600",
    "CODPD0000",
    "PAPEP0000",
    "PAPPD0000",
    "NY0303000",
    "MN0271100",
    "MI8234900",
    "MA0130100",
    "MDBPD0000",
]


In [43]:
agency_subset = agency[agency.index.get_level_values("ori_code").isin(AGENCIES)].copy()
agency_subset


year                       1995                                 1996  \
card                     actual cleared_arrest clearance_rate actual   
ori_code  agency_name                                                  
CA0194200 LOS ANGELES     849.0          482.0       0.567727  711.0   
CA0340400 SACRAMENTO       57.0           41.0       0.719298   43.0   
CA0380100 SAN FRANCISCO    99.0           25.0       0.252525   82.0   
CODPD0000 DENVER           81.0           63.0       0.777778   64.0   
FL0130600 MIAMI           110.0           40.0       0.363636  124.0   
MA0130100 BOSTON           96.0           46.0       0.479167   59.0   
MDBPD0000 BALTIMORE       325.0          149.0       0.458462  330.0   
MI8234900 DETROIT         475.0            0.0       0.000000  428.0   
MN0271100 MINNEAPOLIS      96.0            0.0       0.000000   83.0   
NY0303000 NEW YORK       1177.0          344.0       0.292268  983.0   
PAPEP0000 PHILADELPHIA    432.0          283.0       0.655093  420.0   
PAPPD0000 PITTSBURGH       58.0           48.0       0.827586   47.0   
TX2201200 FORT WORTH      108.0           79.0       0.731481   68.0   
TXDPD0000 DALLAS          276.0          207.0       0.750000  217.0   
TXHPD0000 HOUSTON         316.0          209.0       0.661392  261.0   

year                                                    1997                 \
card                    cleared_arrest clearance_rate actual cleared_arrest   
ori_code  agency_name                                                         
CA0194200 LOS ANGELES            408.0       0.573840  576.0          341.0   
CA0340400 SACRAMENTO              46.0       1.069767   41.0           30.0   
CA0380100 SAN FRANCISCO           47.0       0.573171   59.0           46.0   
CODPD0000 DENVER                  40.0       0.625000   69.0           36.0   
FL0130600 MIAMI                    0.0       0.000000   98.0            0.0   
MA0130100 BOSTON                  37.0       0.627119   43.0           20.0   
MDBPD0000 BALTIMORE              205.0       0.621212  312.0          170.0   
MI8234900 DETROIT                251.0       0.586449  469.0          204.0   
MN0271100 MINNEAPOLIS              0.0       0.000000   58.0           11.0   
NY0303000 NEW YORK               634.0       0.644964  770.0          676.0   
PAPEP0000 PHILADELPHIA           291.0       0.692857  418.0          274.0   
PAPPD0000 PITTSBURGH              31.0       0.659574   50.0           35.0   
TX2201200 FORT WORTH              45.0       0.661765   74.0           16.0   
TXDPD0000 DALLAS                 154.0       0.709677  209.0          154.0   
TXHPD0000 HOUSTON                179.0       0.685824  254.0          166.0   

year                                     1998  ...           2017   2018  \
card                    clearance_rate actual  ... clearance_rate actual   
ori_code  agency_name                          ...                         
CA0194200 LOS ANGELES         0.592014  426.0  ...       0.740214  258.0   
CA0340400 SACRAMENTO          0.731707   31.0  ...       0.538462   36.0   
CA0380100 SAN FRANCISCO       0.779661   58.0  ...       0.482143   46.0   
CODPD0000 DENVER              0.521739   51.0  ...       0.684211   65.0   
FL0130600 MIAMI               0.000000   86.0  ...       0.403846   27.0   
MA0130100 BOSTON              0.465116   34.0  ...       0.438596   56.0   
MDBPD0000 BALTIMORE           0.544872  313.0  ...       0.517544  309.0   
MI8234900 DETROIT             0.434968  430.0  ...       0.518797  263.0   
MN0271100 MINNEAPOLIS         0.189655   58.0  ...       0.666667   31.0   
NY0303000 NEW YORK            0.877922  633.0  ...       0.917808  295.0   
PAPEP0000 PHILADELPHIA        0.655502  338.0  ...       0.443038  351.0   
PAPPD0000 PITTSBURGH          0.700000   36.0  ...       0.654545   57.0   
TX2201200 FORT WORTH          0.216216   64.0  ...       0.676056   54.0   
TXDPD0000 DALLAS              0.736842  252.0  ...       0.79640

In [84]:
agency_5yravg[agency_5yravg.index.get_level_values("ori_code").isin(AGENCIES)]


,year,5_year_avg,2020,change
ori_code,agency_name,,,
CA0194200,LOS ANGELES,0.736397,0.549858,-0.253314
CA0340400,SACRAMENTO,0.580750,0.714286,0.229936
CA0380100,SAN FRANCISCO,0.656727,0.750000,0.142028
CODPD0000,DENVER,0.692525,0.649485,-0.062151
FL0130600,MIAMI,0.600712,0.442623,-0.263170
MA0130100,BOSTON,0.494952,0.293103,-0.407814
MDBPD0000,BALTIMORE,0.387028,0.471572,0.218443
MI8234900,DETROIT,0.409912,0.423780,0.033834
MN0271100,MINNEAPOLIS,0.510512,0.379747,-0.256145
